In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk , string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [187]:
job_df = pd.read_csv('/content/1000_ml_jobs_us.csv')

In [120]:
job_df.head()

,Unnamed: 0,job_posted_date,company_address_locality,company_address_region,company_name,company_website,company_description,job_description_text,seniority_level,job_title
0,0,2024-10-31,Indianapolis,Indiana,Upper Hand,https://upperhand.com,Upper Hand is the leading provider of full-sui...,OverviewUpper Hand is embarking on an exciting...,Internship,Internship - Machine Learning Engineer & Data ...
1,1,2025-03-14,San Francisco,California,Ikigai,https://www.ikigailabs.io,"Built upon years of MIT research, Ikigai is a ...",Company DescriptionThe Ikigai platform unlocks...,Mid-Senior level,Machine Learning Engineer
2,2,2025-04-09,San Jose,CA,Adobe,http://www.adobe.com,Adobe is the global leader in digital media an...,Our CompanyChanging the world through digital ...,Entry level,Machine Learning Engineer
3,3,2025-03-22,Mountain View,California,Waymo,https://waymo.com/careers/,On the journey to be the world's most trusted ...,Waymo is an autonomous driving technology comp...,Entry level,"Machine Learning Engineer, Training"
4,4,2025-03-28,Boston,Massachusetts,HMH,http://www.hmhco.com,We are an adaptive learning company that empow...,Job Title: Machine Learning EngineerLocation: ...,Mid-Senior level,Machine Learning Engineer


In [5]:
job_df.shape

(997, 10)

In [6]:
job_df.isnull().sum()

,0
Unnamed: 0,0
job_posted_date,0
company_address_locality,47
company_address_region,113
company_name,0
company_website,14
company_description,12
job_description_text,1
seniority_level,9
job_title,0


In [10]:
job_df[job_df['company_address_region'].isnull()]

,Unnamed: 0,job_posted_date,company_address_locality,company_address_region,company_name,company_website,company_description,job_description_text,seniority_level,job_title
27,27,2025-03-27,New York,NaN,Adapty.io,https://go.adapty.io/main,Revenue management platform to help your app m...,Adapty is a revenue management platform for mo...,Entry level,ML Engineer/Data Scientist
32,32,2025-04-04,"Provo, UT and Seattle, WA",NaN,Qualtrics,https://www.qualtrics.com/,"At Qualtrics, we create software the world’s b...","At Qualtrics, we create software the world’s b...",Entry level,Machine Learning Engineer
34,34,2025-04-02,NaN,NaN,McKinsey & Company,http://www.mckinsey.com,McKinsey & Company is a global management cons...,"Who You'll Work WithAt McKinsey, our most valu...",Mid-Senior level,Machine Learning Engineer
47,47,2024-11-13,NaN,NaN,"Paradyme, a CATHEXIS Company",NaN,NaN,"OverviewParadyme, a CATHEXIS Company is a rapi...",Entry level,Data Scientist / ML Engineer
49,49,2023-11-29,NaN,NaN,Ritual,https://ritual.net,"Ritual is an open, sovereign execution layer f...","About RitualRitual is building a sovereign, ex...",Entry level,Machine Learning Engineer
...,...,...,...,...,...,...,...,...,...,...
944,944,2025-03-28,Worldwide,NaN,"Pyramid Consulting, Inc",https://pyramidci.com/,"Pyramid Consulting, a global leader in workfor...",Immediate need for a talented Data Scientist. ...,Not Applicable,Data Scientist
945,945,2025-02-21,San Francisco,NaN,Sardine,https://www.sardine.ai/,Sardine is the leading AI risk platform for fr...,Who we are:We are a leader in fraud prevention...,Mid-Senior level,Machine Learning Engineer
956,956,2025-03-27,The Cloud,NaN,Snowflake,http://www.snowflake.com,Snowflake delivers the AI Data Cloud — a globa...,Build the future of the AI Data Cloud. Join th...,Mid-Senior level,Senior Machine Learning Engineer
983,983,2025-04-01,Palo Alto,NaN,BitGo,http://bitgo.com,BitGo is the leading infrastructure provider o...,BitGo is the leading infrastructure provider o...,Mid-Senior level,Software Engineer - Onboarding


In [11]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997 entries, 0 to 996
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                997 non-null    int64 
 1   job_posted_date           997 non-null    object
 2   company_address_locality  950 non-null    object
 3   company_address_region    884 non-null    object
 4   company_name              997 non-null    object
 5   company_website           983 non-null    object
 6   company_description       985 non-null    object
 7   job_description_text      996 non-null    object
 8   seniority_level           988 non-null    object
 9   job_title                 997 non-null    object
dtypes: int64(1), object(9)
memory usage: 78.0+ KB


In [188]:
job_df.drop(['Unnamed: 0','company_address_region'],axis=1,inplace=True)

In [189]:
job_df['company_description'].fillna('',inplace=True)

<ipython-input-189-d8279db4576b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  job_df['company_description'].fillna('',inplace=True)


In [190]:
job_df.dropna(inplace=True)

In [18]:
job_df.head()

,job_posted_date,company_address_locality,company_name,company_website,company_description,job_description_text,seniority_level,job_title
0,2024-10-31,Indianapolis,Upper Hand,https://upperhand.com,Upper Hand is the leading provider of full-sui...,OverviewUpper Hand is embarking on an exciting...,Internship,Internship - Machine Learning Engineer & Data ...
1,2025-03-14,San Francisco,Ikigai,https://www.ikigailabs.io,"Built upon years of MIT research, Ikigai is a ...",Company DescriptionThe Ikigai platform unlocks...,Mid-Senior level,Machine Learning Engineer
2,2025-04-09,San Jose,Adobe,http://www.adobe.com,Adobe is the global leader in digital media an...,Our CompanyChanging the world through digital ...,Entry level,Machine Learning Engineer
3,2025-03-22,Mountain View,Waymo,https://waymo.com/careers/,On the journey to be the world's most trusted ...,Waymo is an autonomous driving technology comp...,Entry level,"Machine Learning Engineer, Training"
4,2025-03-28,Boston,HMH,http://www.hmhco.com,We are an adaptive learning company that empow...,Job Title: Machine Learning EngineerLocation: ...,Mid-Senior level,Machine Learning Engineer


In [124]:
job_df['job_posted_date']=pd.to_datetime(job_df['job_posted_date'])

In [26]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 939 entries, 0 to 996
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   job_posted_date           939 non-null    datetime64[ns]
 1   company_address_locality  939 non-null    object        
 2   company_name              939 non-null    object        
 3   company_website           939 non-null    object        
 4   company_description       939 non-null    object        
 5   job_description_text      939 non-null    object        
 6   seniority_level           939 non-null    object        
 7   job_title                 939 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 98.3+ KB


In [27]:
job_df['job_title'].unique()

array(['Internship - Machine Learning Engineer & Data Science',
       'Machine Learning Engineer', 'Machine Learning Engineer, Training',
       '2025 Intern - Machine Learning Engineer AI/ML',
       'Data Scientist - Healthcare & Clinical Data (Remote)',
       'Machine Learning Software Engineer (L5) - Content and Studio',
       'Machine Learning Engineer (L4/5) - Studio Media Algorithms',
       'Machine Learning Engineer, Applied ML',
       'Software Engineer, Machine Learning',
       'Senior Machine Learning Engineer, NLP',
       'Software Engineer L4, Machine Learning Platform (Metaflow)',
       'Machine Learning Engineer Intern, Summer 2025',
       'Machine Learning Engineer - Early in Career',
       'Senior Data Scientist, Data Science',
       '2025 Intern CV/ML - Machine Learning Engineer',
       'Data Scientist - All Levels', 'ML Engineer/Data Scientist',
       'ML Software Engineer (L4/L5) - Media Algorithms',
       '(General Hire) Machine Learning Engineer Grad

In [28]:
job_df['seniority_level'].unique()

array(['Internship', 'Mid-Senior level', 'Entry level', 'Not Applicable',
       'Associate', 'Executive', 'Director'], dtype=object)

In [125]:
job_df.duplicated().sum()

np.int64(152)

In [191]:
job_df.drop_duplicates(inplace=True)

In [127]:
job_df['job_description_text'][0]

'OverviewUpper Hand is embarking on an exciting journey to harness the power of machine learning, data science, and AI to enhance sports training and management, and we’re looking for an energetic and talented Machine Learning Engineer and Data Scientist to join our AI Data Science Team. As part of our team, you will be instrumental in building and deploying machine learning models while ensuring they align with real business use cases. You will also leverage your expertise in data science to develop data-driven insights and optimize AI-driven initiatives. This role is ideal for someone who thrives in a fast-paced, collaborative environment, loves solving real-world problems through machine learning, and has a passion for communication and presenting solutions.What You’ll DoBuild, deploy, and maintain scalable machine learning models that solve key business challenges.Work closely with external clients and internal cross-functional teams to translate business needs into machine learnin

In [128]:
job_df['company_description'][0]

'Upper Hand is the leading provider of full-suite sports management software that enable businesses to achieve more while doing less.\n\nSports facilities, training academies, and businesses - from small enterprises to large franchises - use Upper Hand’s integrated software and payments platform to operate their business more efficiently, increase customer satisfaction, and leverage real-time data for better decision-making. \n\nIts product suite includes features to manage client booking and registration, memberships, staff and facility scheduling, retail, video analysis, and more. For more information on how Upper Hand is reinventing the way business is done in sports and fitness, visit upperhand.com.'

In [192]:
job_df['company_description'][0]+' '+job_df['job_description_text'][0]

'Upper Hand is the leading provider of full-suite sports management software that enable businesses to achieve more while doing less.\n\nSports facilities, training academies, and businesses - from small enterprises to large franchises - use Upper Hand’s integrated software and payments platform to operate their business more efficiently, increase customer satisfaction, and leverage real-time data for better decision-making. \n\nIts product suite includes features to manage client booking and registration, memberships, staff and facility scheduling, retail, video analysis, and more. For more information on how Upper Hand is reinventing the way business is done in sports and fitness, visit upperhand.com. OverviewUpper Hand is embarking on an exciting journey to harness the power of machine learning, data science, and AI to enhance sports training and management, and we’re looking for an energetic and talented Machine Learning Engineer and Data Scientist to join our AI Data Science Team.

In [193]:
job_df['job_descriptions']=job_df['company_description']+' '+job_df['job_description_text']+' '+job_df['seniority_level']

In [194]:
job_df.reset_index(drop='first',inplace=True)

In [139]:
job_df.head()

,job_posted_date,company_address_locality,company_name,company_website,company_description,job_description_text,seniority_level,job_title,job_descriptions
0,2024-10-31,Indianapolis,Upper Hand,https://upperhand.com,Upper Hand is the leading provider of full-sui...,OverviewUpper Hand is embarking on an exciting...,Internship,Internship - Machine Learning Engineer & Data ...,Upper Hand is the leading provider of full-sui...
1,2025-03-14,San Francisco,Ikigai,https://www.ikigailabs.io,"Built upon years of MIT research, Ikigai is a ...",Company DescriptionThe Ikigai platform unlocks...,Mid-Senior level,Machine Learning Engineer,"Built upon years of MIT research, Ikigai is a ..."
2,2025-04-09,San Jose,Adobe,http://www.adobe.com,Adobe is the global leader in digital media an...,Our CompanyChanging the world through digital ...,Entry level,Machine Learning Engineer,Adobe is the global leader in digital media an...
3,2025-03-22,Mountain View,Waymo,https://waymo.com/careers/,On the journey to be the world's most trusted ...,Waymo is an autonomous driving technology comp...,Entry level,"Machine Learning Engineer, Training",On the journey to be the world's most trusted ...
4,2025-03-28,Boston,HMH,http://www.hmhco.com,We are an adaptive learning company that empow...,Job Title: Machine Learning EngineerLocation: ...,Mid-Senior level,Machine Learning Engineer,We are an adaptive learning company that empow...


**For recommendation system  Dataset**

In [140]:
recommand_df = job_df[['job_title','job_descriptions']]

In [141]:
lem = WordNetLemmatizer()

In [142]:
def cleaning(text):
  text = text.lower()
  text = text.translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))
  text = text.strip()
  stop_words = [words for words in text.split() if words not in stopwords.words('english')]
  alpha_numeric = [word for word in stop_words if word.isalnum()]
  lemin = [lem.lemmatize(words) for words in alpha_numeric]
  return ' '.join(lemin)


In [143]:
recommand_df['clean_job_descriptions'] = recommand_df['job_descriptions'].apply(cleaning)

<ipython-input-143-c2bd7a37ff2b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommand_df['clean_job_descriptions'] = recommand_df['job_descriptions'].apply(cleaning)


In [82]:
recommand_df['clean_job_descriptions'][0]

'upper hand leading provider full suite sport management software enable business achieve less sport facility training academy business small enterprise large franchise use upper integrated software payment platform operate business efficiently increase customer satisfaction leverage real time data better decision making product suite includes feature manage client booking registration membership staff facility scheduling retail video analysis information upper hand reinventing way business done sport fitness visit upperhand com overviewupper hand embarking exciting journey harness power machine learning data science ai enhance sport training management looking energetic talented machine learning engineer data scientist join ai data science team part team instrumental building deploying machine learning model ensuring align real business use case also leverage expertise data science develop data driven insight optimize ai driven initiative role ideal someone thrives fast paced collabor

In [144]:
recommand_df.drop(['job_descriptions'],axis=1,inplace=True)

<ipython-input-144-f7f8d61c2685>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommand_df.drop(['job_descriptions'],axis=1,inplace=True)


In [145]:
recommand_df.rename(columns={'clean_job_descriptions':'job_descriptions'},inplace=True)

<ipython-input-145-8871d6852157>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommand_df.rename(columns={'clean_job_descriptions':'job_descriptions'},inplace=True)


In [146]:
vectorizer = TfidfVectorizer(stop_words='english')

In [147]:
tfidf_vector=vectorizer.fit_transform(recommand_df['job_descriptions'])

In [89]:
tfidf_vector.shape

(939, 16236)

In [95]:
list(vectorizer.get_feature_names_out())

['00',
 '000',
 '000above',
 '000bay',
 '000ca',
 '000grubhub',
 '000individual',
 '000knetflix',
 '000location',
 '000new',
 '000our',
 '000salary',
 '000this',
 '000what',
 '000x',
 '000zone',
 '0024',
 '00in',
 '00maximum',
 '00pay',
 '00the',
 '00this',
 '00what',
 '01',
 '018023the',
 '01active',
 '020',
 '0200',
 '0203',
 '032025',
 '039location',
 '04',
 '040',
 '04397',
 '05',
 '06',
 '065',
 '07',
 '070',
 '08',
 '088',
 '10',
 '100',
 '1000',
 '10010',
 '100a',
 '100b',
 '100k',
 '100m',
 '100x',
 '101197',
 '101m',
 '102',
 '104',
 '10424',
 '104500',
 '105',
 '106',
 '106590',
 '106m',
 '107',
 '109',
 '10b',
 '10gige',
 '10k',
 '10m',
 '10x',
 '11',
 '110',
 '111',
 '112',
 '112000',
 '112200',
 '113',
 '113500',
 '114',
 '11444',
 '115',
 '1157',
 '1158',
 '116',
 '116000',
 '117',
 '117000',
 '118',
 '118657',
 '118750',
 '119',
 '119000',
 '11m',
 '11monday',
 '12',
 '120',
 '120000',
 '120k',
 '122',
 '123',
 '124',
 '125',
 '125k',
 '126',
 '126000',
 '127',
 '127500'

In [148]:
similarity_job = cosine_similarity(tfidf_vector)

In [97]:
similarity_job

array([[1.        , 0.1569787 , 0.10566603, ..., 0.13505681, 0.12265155,
        0.08674672],
       [0.1569787 , 1.        , 0.1064022 , ..., 0.05073359, 0.11640638,
        0.06084083],
       [0.10566603, 0.1064022 , 1.        , ..., 0.07569052, 0.11812707,
        0.06159042],
       ...,
       [0.13505681, 0.05073359, 0.07569052, ..., 1.        , 0.06104249,
        0.03550669],
       [0.12265155, 0.11640638, 0.11812707, ..., 0.06104249, 1.        ,
        0.05185301],
       [0.08674672, 0.06084083, 0.06159042, ..., 0.03550669, 0.05185301,
        1.        ]])

In [231]:
recommand_df['job_title'].nunique()

431

**Note :** Job_df is copy in test_job_data to drop job_descriptions , company_description

In [197]:
test_job_data=job_df

In [198]:
test_job_data.drop(['job_descriptions','company_description'],axis=1,inplace=True)

**Note :** Most of the time they give same job_title don't worry the job_title will be same but the company are different

In [226]:
def job_recommand(job_title):
  index = test_job_data[test_job_data['job_title']==job_title].index[0]
  similarity = sorted(list(enumerate(similarity_job[index])),key=lambda x:x[1],reverse=True)[1:6]
  job_data = []
  for i in similarity:
    temp = test_job_data.iloc[i[0]]
    job_data.append(temp)
  return job_data


In [227]:
job_recommand('Data Scientist')

[job_posted_date                                                    2025-03-26
 company_address_locality                                             San Jose
 company_name                                                            Adobe
 company_website                                          http://www.adobe.com
 job_description_text        Our CompanyChanging the world through digital ...
 seniority_level                                              Mid-Senior level
 job_title                   Machine Learning Engineer - Marketing Mix Mode...
 Name: 712, dtype: object,
 job_posted_date                                                    2025-04-02
 company_address_locality                                          Los Angeles
 company_name                                                           TikTok
 company_website                          https://www.tiktok.com/about?lang=en
 job_description_text        ResponsibilitiesAbout the TeamThe Marketing Da...
 seniority_level         

**Second Option :** NearestNeighbors

In [228]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='cosine')
nn.fit(tfidf_vector)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [229]:
def job_recommand_nn(job_title):
  index=test_job_data[test_job_data['job_title']==job_title].index[0]
  distance,indices = nn.kneighbors(tfidf_vector[index],n_neighbors=6)
  return test_job_data['job_title'].iloc[indices[0][1:6]]

In [230]:
job_recommand_nn('Data Scientist')

,job_title
712,Machine Learning Engineer - Marketing Mix Mode...
731,"Data Scientist, Marketing Insights and Analytics"
624,Machine Learning Engineer
481,Machine Learning Engineer
331,Data Scientist


**Note :** Both having same result you can use any of them

In [232]:
import pickle
pickle.dump(test_job_data,open('job_data.pkl','wb'))
pickle.dump(similarity_job,open('similarity_job.pkl','wb'))